In [1]:
import requests, json
import time
#from time import strftime
import datetime
import numpy as np
import re
import xmltodict
import pprint# as pp
#import process_command
#print(dir(process_command))

## Internal functions

In [2]:
baseurl="https://v5.vbb.transport.rest/"
urlending="&accept=application/x-ndjson"
def openWebsite(adress):
    print(adress)
    r = requests.get(adress)
    return r.text

In [3]:
def prettyPrint(text):
    pp = pprint.PrettyPrinter(indent=1)
    pp.pprint(text)

In [4]:
def process_command_vbb(command, param):  
    def furtherParamString(param):#,listToDelete):
        ret=""
        for key in param.keys():
            ret+= "&"+str(key)+"="+str(param[key])
        return ret
    beforeId=command
    behindId=furtherParamString(param)
    return {"beforeId":beforeId, "behindId":behindId}

In [5]:
def createAdress_vbb(command, param):
    commands = process_command_vbb(command, param)
    #accessId ="felix-fauer-8b71-1705950c2589"
    adress = baseurl + commands['beforeId'] + "/" + commands['behindId']+"&accept=application/x-ndjson"
    #print(adress)
    return adress

In [6]:
print("https://v5.vbb.transport.rest/stops/900000013102/departures?when=tomorrow%206pm&results=3&accept=application/x-ndjson")

https://v5.vbb.transport.rest/stops/900000013102/departures?when=tomorrow%206pm&results=3&accept=application/x-ndjson


In [7]:
def responseToDict(response, return_json=True):
    if return_json:
        return json.loads(response)
    else:
        return xmltodict.parse(response)

In [8]:
def getData(command, param={}):
    response_xml = openWebsite(createAdress_vbb(command, param))
    return responseToDict(response_xml)    
#getData('locations?', {"query":"Tauernallee","maxNo":1,"poi":"false","addresses":"false"})

In [9]:
def nameToExtStation(name):
    response = getData('locations?', {"query":name,"maxNo":1,"poi":"false","addresses":"false"})
    return response[0]['location']['id']#['@extId']
#nameToExtStation("Tauernallee")

In [10]:
def nextDeparturesAtStop(name=False, ext=False, maxNo=3):
    if not ext:
        if name:
            ext=nameToExtStation(name)
        else:
            print("ERROR. You must provide name or ext.")
        
    command, param ='stops/'+str(ext)+'/'+'departures?', {'results':maxNo}
    response = getData(command, param)
    return response
#nextDeparturesAtStop("Tauernallee")

In [11]:
def stopsOnLine(name):
    pass

In [12]:
def getLinesAtStop(name=False, ext=False):
    if not ext:
        ext=nameToExtStation(name)
    else:
        pass


# Try and debug

In [13]:
nameToExtStation("U Alt-Mariendorf")

https://v5.vbb.transport.rest/locations?/&query=U Alt-Mariendorf&maxNo=1&poi=false&addresses=false&accept=application/x-ndjson


'900070301'

In [14]:
ext = 900070401
nextDep = nextDeparturesAtStop(ext=900070401, maxNo=4)
now = datetime.datetime.now()
print("now: ", now)

for i in range(4):
    print(i)
    time_str = nextDep[i]["when"]
    iTime = datetime.datetime.strptime(time_str[0:19],
                                        '%Y-%m-%dT%H:%M:%S')
    print("time:", iTime)
    diffTime = iTime-now
    print("diff:", diffTime)

    if diffTime.seconds >24*60*60/2:
        diffMin = (24*60*60-diffTime.seconds)//60
    else:
        diffMin = diffTime.seconds//60


    print("print", diffMin)
#datetime(time_str)

https://v5.vbb.transport.rest/stops/900070401/departures?/&results=4&accept=application/x-ndjson
now:  2022-09-18 10:37:11.931031
0
time: 2022-09-18 10:37:00
diff: -1 day, 23:59:48.068969
print 0
1
time: 2022-09-18 10:40:00
diff: 0:02:48.068969
print 2
2
time: 2022-09-18 10:46:00
diff: 0:08:48.068969
print 8
3
time: 2022-09-18 10:47:00
diff: 0:09:48.068969
print 9


## Usable functions:

In [14]:
def print_nextXDeparturesAtStop(name, maxNo=3):
    response = nextDeparturesAtStop(name, maxNo)
    print("Next departures at "+str(response[1]['stop']['name'])+":\n")
    print("   Line        Destination                     Time                  (intern data)")
    print("   ----        -----------                     ----")
    for dep in response:
        if type(dep['delay'])==int:
            print("   "+dep['line']['id'].lstrip().ljust(8)+"    "+dep['direction'].ljust(30)[0:30]+"  "+
                 dep['plannedWhen'][11:16]+" ("+dep['when'][11:16]+", d="+str(dep['delay']/60)+")")
        else:
            print("   "+dep['line']['id'].lstrip().ljust(8)+"    "+dep['direction'].ljust(30)[0:30]+"  "+
                 dep['plannedWhen'][11:16]+" ("+dep['when'][11:16]+")"+"".ljust(9))
print_nextXDeparturesAtStop("Zoolo",10)

TypeError: can only concatenate str (not "int") to str

In [14]:
#from numpy import genfromtxt
import numpy as np
lines = np.genfromtxt('lines.csv', delimiter=',',dtype=str)
operators = np.genfromtxt('operators.csv', delimiter=',',dtype=str)

for x in range(lines.shape[0]):
    for y in range(lines.shape[1]):
        if type(lines[x,y]==str):
            lines[x,y] = lines[x,y]. replace('"','')

#my_data_bvg = my_data[my_data[:,2]==796,2]
# bvg 796, sbahn 1, regio berlin 108
# operator in agency.txt
# 

In [58]:
def get_line_id(name, route_type='700', means=False):
    if not means:
        if "S" in name:
            route_type='109'
        elif "U" in name:
            route_type='400'
        elif ("RE" in name) or ("RB" in name):
            route_type='100'
        elif "F" in name:
            route_type='1000'
        elif "M" in name: # metro tram
            if len(name)==2 or name in ('M10','M13','M17'):
                route_type='900'
        elif ("N" in name) or ("X" in name):
            route_type='700'
        elif len(name)==2:
            route_type='900'
    elif means=="bus":
        route_type='700'
    elif means=="s":
        route_type='109'
    elif means=="u":
        route_type='400'
    elif means=="regio":
        route_type='100'
    elif means=="tram":
        route_type='900'
    elif means=="ferry":
        route_type='1000'

    idx = np.where(lines[:,2]==name)
    ids = lines[idx,:][0]
    if ids.shape[0]==1:
        return(ids[0,0])
    else:
        row = np.where(ids[:,4]==route_type)
        remain=ids[row,0][0]
        return(remain[0])

lineID = get_line_id("F10")
print("returned", lineID)

returned 17429_1000


In [89]:
def get_stops_of_line(line_id):
        response = openWebsite(baseurl+'lines/'+line_id+"/?"+urlending)
        #response = getData('lines/'+line_id+"/?")
        return responseToDict(response)['variants'][0]['stops']

lineID = get_line_id("179")
stopIDs = get_stops_of_line(lineID)
print(stopIDs)

https://v5.vbb.transport.rest/lines/17337_700/?&accept=application/x-ndjson
['070101003392', '070101004361', '070101002995', '070101002592', '070101003379', '070101003380', '070101002496', '070101002497', '070101002498', '070101002485', '070101002386', '070101004025', '070101002387', '070101004193', '070101003568']


In [93]:
for istop in range(3):#len(stopIDs)):
    deps = nextDeparturesAtStop(ext=stopIDs[istop], maxNo=3)

https://v5.vbb.transport.rest/stops/070101003392/departures?/&results=3&accept=application/x-ndjson
https://v5.vbb.transport.rest/stops/070101004361/departures?/&results=3&accept=application/x-ndjson
https://v5.vbb.transport.rest/stops/070101002995/departures?/&results=3&accept=application/x-ndjson


In [94]:
deps

{'isHafasError': True,
 'request': '{"lang":"de","svcReqL":[{"meth":"StationBoard","req":{"type":"DEP","date":"20220227","time":"204103","stbLoc":{"type":"S","lid":"A=1@L=001002995@"},"jnyFltrL":[{"type":"PROD","mode":"INC","value":"127"}],"dur":10,"maxJny":3}}],"client":{"type":"WEB","id":"VBB","name":"VBB WebApp","l":"vs_webapp_vbb"},"ver":"1.45","auth":{"type":"AID","aid":"hafas-vbb-webapp"}}',
 'url': 'https://fahrinfo.vbb.de/bin/mgate.exe?',
 'statusCode': 400,
 'isClient': True,
 'code': 'INVALID_REQUEST',
 'message': 'location/stop not found',
 'hafasErrorCode': 'LOCATION',
 'hafasErrorMessage': 'HCI Service: location missing or invalid',
 'error': True,
 'msg': 'HAFAS error: location/stop not found'}